In [1]:
%%capture

# libraries for data manipulation
import pandas as pd
import numpy as np

# libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# import train_test_split from sklearn
from sklearn.model_selection import train_test_split

# Import models from scikit learn module:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.validation import check_is_fitted
from sklearn.model_selection import GridSearchCV

# libraries others models 
import xgboost as xgb
import lightgbm as lgb

# Libraries for Experiment Tracking
import mlflow

# Libraries for Model Training and Evaluation
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.utils.validation import check_is_fitted

# libraries for file management
import os
import shutil
from tqdm import tqdm


os.chdir("../")

from  src import  data_utils, train_log_experiments

# autoreload modules
%load_ext autoreload
%autoreload 2

from imblearn.over_sampling import (
    SMOTE,
    BorderlineSMOTE,
    ADASYN,
    KMeansSMOTE
)
from imblearn.combine import SMOTEENN, SMOTETomek

In [2]:
os.getcwd()

'c:\\Users\\crist\\Documents\\anyoneai\\final_d\\finalprojectApiAnyoneAI\\training'

In [2]:
# load csv files
X_train = pd.read_csv("dataset/processed/X_train.csv")
y_train = pd.read_csv("dataset/processed/y_train.csv")
X_test = pd.read_csv("dataset/processed/X_test.csv")
y_test = pd.read_csv("dataset/processed/y_test.csv")


In [8]:
# to numpy
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [10]:
# Generar las diferentes versiones de X_train y y_train
X_train_variants = {}
y_train_variants = {}

# Definir las variantes de oversampling
oversamplers = {
    'SMOTE': SMOTE(),
    'BorderlineSMOTE': BorderlineSMOTE(),
    'ADASYN': ADASYN(),
    # 'KMeansSMOTE': KMeansSMOTE()
}

# Definir las variantes de combinación de oversampling y undersampling
combined_samplers = {
    'SMOTEENN': SMOTEENN(),
    'SMOTETomek': SMOTETomek()
}

# add the original data to dict
X_train_variants['original'] = X_train
y_train_variants['original'] = y_train


In [11]:
for oversampler_name, oversampler in tqdm(oversamplers.items(), desc='Oversampling variants'):
    try:
        X_train_os, y_train_os = oversampler.fit_resample(X_train, y_train)
        X_train_variants[oversampler_name] = X_train_os
        y_train_variants[oversampler_name] = y_train_os
    except RuntimeError as e:
        print(f"Error encountered with {oversampler_name}: {e}")
        continue


Oversampling variants: 100%|██████████| 3/3 [01:09<00:00, 23.32s/it]


In [12]:
for oversampler_name, combined_samplers in tqdm(combined_samplers.items(), desc='Oversampling combined variants'):
    try:
        X_train_os, y_train_os = combined_samplers.fit_resample(X_train, y_train)
        X_train_variants[oversampler_name] = X_train_os
        y_train_variants[oversampler_name] = y_train_os
    except RuntimeError as e:
        print(f"Error encountered with {oversampler_name}: {e}")
        continue


Oversampling combined variants: 100%|██████████| 2/2 [06:35<00:00, 197.78s/it]


In [13]:
# Elige un valor para el estado aleatorio.
random_state = 42

# Llama a la función.
train_log_experiments.train_and_log_logistic_regression_baseline(
    X_train,
    y_train,
    X_test,
    y_test,
    random_state=42,
    solver='saga',
    max_iter=1000,
    C=0.5,
    penalty='l1',
    run_name="Baseline")

C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\crist\miniconda3\envs\gpu_base\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


ROC AUC on validation set: 0.6311


In [14]:
for variant_name, X_train_variant in tqdm(X_train_variants.items()):
    y_train_variant = y_train_variants[variant_name]

    train_log_experiments.train_and_log_logistic_regression_baseline(
    X_train_variant,
    y_train_variant,
    X_test,
    y_test,
    random_state=42,
    solver='saga',
    max_iter=5000,
    C=0.5,
    penalty='l1',
    run_name="Baseline_" + variant_name)

 17%|█▋        | 1/6 [24:36<2:03:00, 1476.12s/it]

ROC AUC on validation set: 0.6312


 33%|███▎      | 2/6 [1:29:52<3:14:06, 2911.57s/it]

ROC AUC on validation set: 0.6259


In [ ]:
for variant_name, X_train_variant in X_train_variants.items():
    y_train_variant = y_train_variants[variant_name]
    # print shaple
    print(f"X_train_variant: {X_train_variant.shape}")
    print(f"y_train_variant: {y_train_variant.shape}")
    

X_train_variant: (40000, 1773)
y_train_variant: (40000, 1)
X_train_variant: (59134, 1773)
y_train_variant: (59134,)
X_train_variant: (59134, 1773)
y_train_variant: (59134,)
X_train_variant: (61076, 1773)
y_train_variant: (61076,)
X_train_variant: (32463, 1773)
y_train_variant: (32463,)
X_train_variant: (58386, 1773)
y_train_variant: (58386,)


In [ ]:
def train_and_log_model(X_train, y_train, X_test, y_test, n_estimators, max_depth, random_state, run_name="Baseline"):
    # Start an MLflow run with the specified run_name
    with mlflow.start_run(run_name=run_name):
        # Initialize and train the Random Forest Classifier with the given hyperparameters
        rf_clf = RandomForestClassifier(n_estimators=n_estimators,
                                        max_depth=max_depth,
                                        random_state=random_state)
        rf_clf.fit(X_train, y_train)

        # Log the model's hyperparameters
        mlflow.log_param("Model", run_name)
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("random_state", random_state)

        # Calculate and log the accuracy metric for the training dataset
        train_preds = rf_clf.predict(X_train)
        train_accuracy = accuracy_score(y_train, train_preds)
        mlflow.log_metric("train_accuracy", train_accuracy)

        # Calculate and log the ROC AUC metric for the training dataset
        train_probs = rf_clf.predict_proba(X_train)[:, 1]
        train_roc_auc = roc_auc_score(y_train, train_probs)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Log the trained model using mlflow.sklearn.log_model()
        mlflow.sklearn.log_model(rf_clf, "random_forest_classifier")

        # Calculate and log the ROC AUC metric for the validation dataset
        val_probs = rf_clf.predict_proba(X_test)[:, 1]
        val_roc_auc = roc_auc_score(y_test, val_probs)
        mlflow.log_metric("val_roc_auc", val_roc_auc)

        # Print the ROC AUC score for the validation dataset
        print(f"ROC AUC en el conjunto de validación: {val_roc_auc:.4f}")

In [16]:
for variant_name, X_train_variant in tqdm(X_train_variants.items()):
    y_train_variant = y_train_variants[variant_name]

    train_log_experiments.train_and_log_model_rf(
    X_train_variant,
    y_train_variant,
    X_test, y_test,
    n_estimators=60,
    max_depth=5,
    random_state=42,
    run_name="RandomForestClassifier_" + variant_name)

 17%|█▋        | 1/6 [00:05<00:25,  5.18s/it]

ROC AUC en el conjunto de validación: 0.6123


 33%|███▎      | 2/6 [01:07<02:34, 38.52s/it]

ROC AUC en el conjunto de validación: 0.6007


 50%|█████     | 3/6 [01:19<01:20, 26.81s/it]

...

ROC AUC en el conjunto de validación: 0.5958


 83%|████████▎ | 5/6 [01:40<00:16, 16.51s/it]

ROC AUC en el conjunto de validación: 0.5860


100%|██████████| 6/6 [01:52<00:00, 18.78s/it]

ROC AUC en el conjunto de validación: 0.5803


In [17]:
train_log_experiments.train_and_log_model_rf(X_train, y_train, X_test, y_test, n_estimators=60, max_depth=5, random_state=42, run_name="RandomForestClassifier")

ROC AUC en el conjunto de validación: 0.6123


In [ ]:
def train_and_log_model(X_train, y_train, X_test, y_val, n_iter, cv, random_state,
                        n_estimators_range, max_depth_range, min_samples_split_values,
                        min_samples_leaf_values, bootstrap_values, run_name="RandomizedSearchCV"):
    
    param_grid = {
        "n_estimators": n_estimators_range,
        "max_depth": max_depth_range,
        "min_samples_split": min_samples_split_values,
        "min_samples_leaf": min_samples_leaf_values,
        "bootstrap": bootstrap_values,
    }

    rf_random = RandomizedSearchCV(
        estimator=RandomForestClassifier(),
        param_distributions=param_grid,
        n_iter=n_iter,
        cv=cv,
        verbose=2,
        random_state=random_state,
    )

    with mlflow.start_run(run_name=run_name):
        rf_random.fit(X_train, y_train)

        best_params = rf_random.best_params_
        best_rf_clf = rf_random.best_estimator_

        # Log parameters
        mlflow.log_param("Model", run_name)
        for param, value in best_params.items():
            mlflow.log_param(param, value)

        # Calcular y registrar la métrica de precisión
        train_preds = best_rf_clf.predict(X_train)
        train_accuracy = accuracy_score(y_train, train_preds)
        mlflow.log_metric("train_accuracy", train_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de entrenamiento
        train_probs = best_rf_clf.predict_proba(X_train)[:, 1]
        train_roc_auc = roc_auc_score(y_train, train_probs)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Registrar el modelo
        mlflow.sklearn.log_model(best_rf_clf, "random_forest_classifier")

        # Calcular y registrar el ROC AUC para el conjunto de validación
        val_probs = best_rf_clf.predict_proba(X_test)[:, 1]
        val_roc_auc = roc_auc_score(y_val, val_probs)
        mlflow.log_metric("val_roc_auc", val_roc_auc)


In [ ]:
n_estimators_range = [int(x) for x in np.linspace(start=200, stop=2000, num=50)]
max_depth_range = [int(x) for x in np.linspace(10, 50, num=5)]
min_samples_split_values = [2, 5]
min_samples_leaf_values = [1, 2]
bootstrap_values = [True]

train_and_log_model(X_train, y_train, X_test, y_test, n_iter=2, cv=2, random_state=42,
                    n_estimators_range=n_estimators_range, max_depth_range=max_depth_range,
                    min_samples_split_values=min_samples_split_values, min_samples_leaf_values=min_samples_leaf_values,
                    bootstrap_values=bootstrap_values)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=273; total time=  23.8s


C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=273; total time=  21.5s


C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END bootstrap=True, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=1485; total time= 4.6min


C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END bootstrap=True, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=1485; total time= 4.3min


C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [18]:
def train_and_log_model(X_train, y_train, X_test, y_val, n_iter, cv, random_state,
                        n_estimators_range, max_depth_range, min_samples_split_values,
                        min_samples_leaf_values, bootstrap_values, run_name="RandomizedSearchCV"):
    
    param_grid = {
        "n_estimators": n_estimators_range,
        "max_depth": max_depth_range,
        "min_samples_split": min_samples_split_values,
        "min_samples_leaf": min_samples_leaf_values,
        "bootstrap": bootstrap_values,
    }

    rf_random = RandomizedSearchCV(
        estimator=RandomForestClassifier(),
        param_distributions=param_grid,
        n_iter=n_iter,
        cv=cv,
        verbose=2,
        random_state=random_state,
    )

    with mlflow.start_run(run_name=run_name):
        rf_random.fit(X_train, y_train)

        best_params = rf_random.best_params_
        best_rf_clf = rf_random.best_estimator_

        # Log parameters
        mlflow.log_param("Model", run_name)
        for param, value in best_params.items():
            mlflow.log_param(param, value)

        # Calcular y registrar la métrica de precisión
        train_preds = best_rf_clf.predict(X_train)
        train_accuracy = accuracy_score(y_train, train_preds)
        mlflow.log_metric("train_accuracy", train_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de entrenamiento
        train_probs = best_rf_clf.predict_proba(X_train)[:, 1]
        train_roc_auc = roc_auc_score(y_train, train_probs)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Registrar el modelo
        mlflow.sklearn.log_model(best_rf_clf, "random_forest_classifier")

        # Calcular y registrar el ROC AUC para el conjunto de validación
        val_probs = best_rf_clf.predict_proba(X_test)[:, 1]
        val_roc_auc = roc_auc_score(y_val, val_probs)
        mlflow.log_metric("val_roc_auc", val_roc_auc)

        # Log feature importance
        feature_importance = best_rf_clf.feature_importances_
        for i, importance in enumerate(feature_importance):
            mlflow.log_metric(f"feature_{i}_importance", importance)


In [19]:
# Ejemplo de uso de la función
n_estimators_range = [int(x) for x in np.linspace(start=200, stop=2000, num=50)]
max_depth_range = [int(x) for x in np.linspace(10, 50, num=5)]
min_samples_split_values = [2, 5]
min_samples_leaf_values = [1, 2]
bootstrap_values = [True]

train_and_log_model(X_train, y_train, X_test, y_test, n_iter=2, cv=2, random_state=42,
                    n_estimators_range=n_estimators_range, max_depth_range=max_depth_range,
                    min_samples_split_values=min_samples_split_values, min_samples_leaf_values=min_samples_leaf_values,
                    bootstrap_values=bootstrap_values)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=273; total time=  19.5s


C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=273; total time=  19.3s


C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END bootstrap=True, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=1485; total time= 3.9min


C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END bootstrap=True, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=1485; total time= 3.5min


C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [23]:
# X_train, X_val, y_train, y_val
params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "num_leaves": 40,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
}
for variant_name, X_train_variant in tqdm(X_train_variants.items()):
    y_train_variant = y_train_variants[variant_name]

    train_log_experiments.train_and_log_lightgbm(
    X_train_variant,
    y_train_variant,
    X_test, y_test,
    params,
    run_name="lgbm_" + variant_name)

  0%|          | 0/6 [00:00<?, ?it/s]c:\Users\crist\miniconda3\envs\gpu_base\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[LightGBM] [Info] Number of positive: 10433, number of negative: 29567
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.112841 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2743
[LightGBM] [Info] Number of data points in the train set: 40000, number of used features: 1000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.260825 -> initscore=-1.041685
[LightGBM] [Info] Start training from score -1.041685
[1]	valid_0's auc: 0.603941
[2]	valid_0's auc: 0.612068
[3]	valid_0's auc: 0.612466
[4]	valid_0's auc: 0.613814
[5]	valid_0's auc: 0.616551
[6]	valid_0's auc: 0.618908
[7]	valid_0's auc: 0.619217
[8]	valid_0's auc: 0.619632
[9]	valid_0's auc: 0.620027
[10]	valid_0's auc: 0.620787
[11]	valid_0's auc: 0.620687
[12]	valid_0's auc: 0.621169
[13]	valid_0's auc: 0.621416
[14]	valid_0's auc: 0.622624
[15]	valid_0's auc: 0.623001
[16]	valid_

 17%|█▋        | 1/6 [00:07<00:37,  7.48s/it]

ROC AUC en el conjunto de validación: 0.6423
[LightGBM] [Info] Number of positive: 29567, number of negative: 29567
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.092687 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29149
[LightGBM] [Info] Number of data points in the train set: 59134, number of used features: 1198


c:\Users\crist\miniconda3\envs\gpu_base\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


...

ROC AUC en el conjunto de validación: 0.6278
[LightGBM] [Info] Number of positive: 29193, number of negative: 29193
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.082742 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28994
[LightGBM] [Info] Number of data points in the train set: 58386, number of used features: 1198


c:\Users\crist\miniconda3\envs\gpu_base\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[1]	valid_0's auc: 0.600728
[2]	valid_0's auc: 0.60141
[3]	valid_0's auc: 0.606598
[4]	valid_0's auc: 0.607255
[5]	valid_0's auc: 0.609717
[6]	valid_0's auc: 0.610724
[7]	valid_0's auc: 0.611303
[8]	valid_0's auc: 0.611208
[9]	valid_0's auc: 0.612255
[10]	valid_0's auc: 0.612317
[11]	valid_0's auc: 0.613609
[12]	valid_0's auc: 0.613808
[13]	valid_0's auc: 0.614272
[14]	valid_0's auc: 0.615007
[15]	valid_0's auc: 0.616118
[16]	valid_0's auc: 0.615643
[17]	valid_0's auc: 0.616397
[18]	valid_0's auc: 0.616468
[19]	valid_0's auc: 0.617036
[20]	valid_0's auc: 0.61735
[21]	valid_0's auc: 0.617711
[22]	valid_0's auc: 0.617558
[23]	valid_0's auc: 0.617729
[24]	valid_0's auc: 0.617337
[25]	valid_0's auc: 0.617876
[26]	valid_0's auc: 0.618369
[27]	valid_0's auc: 0.618636
[28]	valid_0's auc: 0.618228
[29]	valid_0's auc: 0.618423
[30]	valid_0's auc: 0.618808
[31]	valid_0's auc: 0.6191
[32]	valid_0's auc: 0.619864
[33]	v

100%|██████████| 6/6 [00:42<00:00,  7.04s/it]

ROC AUC en el conjunto de validación: 0.6348


In [26]:
import random
from random import randint, uniform

param_dist = {
    'boosting_type': ['gbdt', 'dart', 'goss'], 
    'num_leaves': [20, 30, 40, 50],
    'learning_rate': [0.01, 0.05, 0.1],
    'feature_fraction': [0.7, 0.8, 0.9, 1.0],
}


fixed_params = {
    "objective": "binary",
    "metric": "auc"
}

best_score = 0
best_params = None

num_iterations = 100  # O el número de iteraciones que desees

for _ in range(num_iterations):
    
    params = {**fixed_params, 
              'boosting_type': random.choice(param_dist['boosting_type']), 
              'num_leaves': random.choice(param_dist['num_leaves']), 
              'learning_rate': random.choice(param_dist['learning_rate']), 
              'feature_fraction': random.choice(param_dist['feature_fraction'])}
    
    # En este punto asumimos que tu función retorna el roc_auc_score en el conjunto de validación
    # roc_auc_val = train_and_log_lightgbm(X_train, y_train, X_test, y_test, params)

    for variant_name, X_train_variant in tqdm(X_train_variants.items()):
        y_train_variant = y_train_variants[variant_name]

        train_log_experiments.train_and_log_lightgbm(
        X_train_variant,
        y_train_variant,
        X_test, y_test,
        params,
        run_name="lgbm_custom_" + variant_name)



  0%|          | 0/6 [00:00<?, ?it/s]c:\Users\crist\miniconda3\envs\gpu_base\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[LightGBM] [Info] Number of positive: 10433, number of negative: 29567
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042825 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2743
[LightGBM] [Info] Number of data points in the train set: 40000, number of used features: 1000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.260825 -> initscore=-1.041685
[LightGBM] [Info] Start training from score -1.041685
[1]	valid_0's auc: 0.570207
[2]	valid_0's auc: 0.58968
[3]	valid_0's auc: 0.591559
[4]	valid_0's auc: 0.610803
[5]	valid_0's auc: 0.611896
[6]	valid_0's auc: 0.619411
[7]	valid_0's auc: 0.617007
[8]	valid_0's auc: 0.617239
[9]	valid_0's auc: 0.620401
[10]	valid_0's auc: 0.620404
[11]	valid_0's auc: 0.622895
[12]	valid_0's auc: 0.622581
[13]	valid_0's auc: 0.624732
[14]	valid_0's auc: 0.624807
[15]	valid_0's auc: 0.624783
[16]	valid_0

 17%|█▋        | 1/6 [00:09<00:47,  9.59s/it]

ROC AUC en el conjunto de validación: 0.6407
[LightGBM] [Info] Number of positive: 29567, number of negative: 29567
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.134848 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29149
[LightGBM] [Info] Number of data points in the train set: 59134, number of used features: 1198


c:\Users\crist\miniconda3\envs\gpu_base\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


...

ROC AUC en el conjunto de validación: 0.6248
[LightGBM] [Info] Number of positive: 29193, number of negative: 29193
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.078533 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28994
[LightGBM] [Info] Number of data points in the train set: 58386, number of used features: 1198
[LightGBM] [Info] Using GOSS


c:\Users\crist\miniconda3\envs\gpu_base\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[1]	valid_0's auc: 0.602258
[2]	valid_0's auc: 0.607144
[3]	valid_0's auc: 0.607948
[4]	valid_0's auc: 0.608149
[5]	valid_0's auc: 0.609839
[6]	valid_0's auc: 0.611473
[7]	valid_0's auc: 0.611448
[8]	valid_0's auc: 0.613904
[9]	valid_0's auc: 0.614457
[10]	valid_0's auc: 0.615702
[11]	valid_0's auc: 0.6168
[12]	valid_0's auc: 0.617204
[13]	valid_0's auc: 0.617303
[14]	valid_0's auc: 0.616008
[15]	valid_0's auc: 0.616934
[16]	valid_0's auc: 0.616465
[17]	valid_0's auc: 0.617426
[18]	valid_0's auc: 0.618677
[19]	valid_0's auc: 0.619714
[20]	valid_0's auc: 0.620026
[21]	valid_0's auc: 0.619694
[22]	valid_0's auc: 0.619932
[23]	valid_0's auc: 0.619474
[24]	valid_0's auc: 0.619865
[25]	valid_0's auc: 0.620817
[26]	valid_0's auc: 0.620974
[27]	valid_0's auc: 0.621231
[28]	valid_0's auc: 0.622475
[29]	valid_0's auc: 0.623437
[30]	valid_0's auc: 0.62495
[31]	valid_0's auc: 0.625559
[32]	valid_0's auc: 0.626384
[33]	

100%|██████████| 6/6 [00:56<00:00,  9.50s/it]

ROC AUC en el conjunto de validación: 0.6345


In [28]:
%%capture
# Ejemplo de uso de la función
params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "max_depth": 10,
    "eta": 0.3,
    "silent": 1,
}

train_log_experiments.train_and_log_xgboost(X_train, y_train, X_test, y_test, params)

In [33]:
# Código de búsqueda de hiperparámetros y ejecución de train_and_log_xgboost
param_dist = {
    'boosting_type': ['gbtree', 'dart', 'gblinear'], 
    'num_leaves': [20, 30, 40, 50],
    'learning_rate': [0.01, 0.05, 0.1],
    'feature_fraction': [0.7, 0.8, 0.9, 1.0],
}

fixed_params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
}

best_score = 0
best_params = None

num_iterations = 100  # O el número de iteraciones que desees

for _ in range(num_iterations):
    params = {**fixed_params, 
              'boosting_type': random.choice(param_dist['boosting_type']), 
              'num_leaves': random.choice(param_dist['num_leaves']), 
              'learning_rate': random.choice(param_dist['learning_rate']), 
              'feature_fraction': random.choice(param_dist['feature_fraction'])}

    # Entrenar y registrar el modelo XGBoost con los parámetros seleccionados
    # train_log_experiments.train_and_log_xgboost(X_train, y_train, X_test, y_test, params, run_name="XGBoost")
    for variant_name, X_train_variant in tqdm(X_train_variants.items()):
        y_train_variant = y_train_variants[variant_name]

        train_log_experiments.train_and_log_xgboost(
        X_train_variant,
        y_train_variant,
        X_test, y_test,
        params,
        run_name="xgboost_custom_" + variant_name)

  0%|          | 0/6 [00:00<?, ?it/s]

[13:10:36] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "boosting_type", "feature_fraction", "num_leaves" } are not used.

[0]	validation-auc:0.60699
[1]	validation-auc:0.60707
[2]	validation-auc:0.60914
[3]	validation-auc:0.60927
[4]	validation-auc:0.60836
[5]	validation-auc:0.60952
[6]	validation-auc:0.60922
[7]	validation-auc:0.60968
[8]	validation-auc:0.61019
[9]	validation-auc:0.61091


 17%|█▋        | 1/6 [00:09<00:46,  9.36s/it]

ROC AUC en el conjunto de validación: 0.6109
[13:10:58] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "boosting_type", "feature_fraction", "num_leaves" } are not used.

[0]	validation-auc:0.59686
[1]	validation-auc:0.59678
[2]	validation-auc:0.59945
[3]	validation-auc:0.59944
[4]	validation-auc:0.60182
[5]	validation-auc:0.60183
[6]	validation-auc:0.60163
[7]	validation-auc:0.60160
[8]	validation-auc:0.60205
[9]	validation-auc:0.60267


 33%|███▎      | 2/6 [00:33<01:12, 18.09s/it]

...

 67%|██████▋   | 4/6 [00:41<00:21, 10.66s/it]

ROC AUC en el conjunto de validación: 0.6059
[14:56:56] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "boosting_type", "feature_fraction", "num_leaves" } are not used.

[0]	validation-auc:0.59511
[1]	validation-auc:0.60144
[2]	validation-auc:0.60526
[3]	validation-auc:0.60450
[4]	validation-auc:0.60453
[5]	validation-auc:0.60672
[6]	validation-auc:0.60622
[7]	validation-auc:0.60692
[8]	validation-auc:0.60752
[9]	validation-auc:0.60769


 83%|████████▎ | 5/6 [00:49<00:09,  9.73s/it]

ROC AUC en el conjunto de validación: 0.6077
[14:57:05] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "boosting_type", "feature_fraction", "num_leaves" } are not used.

[0]	validation-auc:0.60017
[1]	validation-auc:0.60446
[2]	validation-auc:0.60581
[3]	validation-auc:0.60655
[4]	validation-auc:0.60725
[5]	validation-auc:0.60828
[6]	validation-auc:0.61035
[7]	validation-auc:0.61091
[8]	validation-auc:0.60974
[9]	validation-auc:0.61102
ROC AUC en el conjunto de validación: 0.6110


100%|██████████| 6/6 [01:00<00:00, 10.17s/it]


In [39]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import mlflow

def train_and_log_ensemble(X_train, y_train, X_test, y_val, ensemble, run_name):
    with mlflow.start_run(run_name=run_name):

        # Entrenar el modelo ensemble
        ensemble.fit(X_train, y_train)

        # Calcular y registrar la métrica de precisión en el conjunto de entrenamiento
        train_preds = np.round(ensemble.predict(X_train))
        train_accuracy = accuracy_score(y_train, train_preds)
        mlflow.log_metric("train_accuracy", train_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de entrenamiento
        train_probs = ensemble.predict_proba(X_train)[:, 1]
        train_roc_auc = roc_auc_score(y_train, train_probs)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Calcular y registrar la métrica de precisión en el conjunto de validación
        val_preds = np.round(ensemble.predict(X_test))
        val_accuracy = accuracy_score(y_val, val_preds)
        mlflow.log_metric("val_accuracy", val_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de validación
        val_probs = ensemble.predict_proba(X_test)[:, 1]
        val_roc_auc = roc_auc_score(y_val, val_probs)
        mlflow.log_metric("val_roc_auc", val_roc_auc)

        # Registrar el modelo
        mlflow.sklearn.log_model(ensemble, ensemble_name)

        print(f"ROC AUC en el conjunto de validación: {val_roc_auc:.4f}")

        return val_roc_auc

# Definir los modelos individuales
logreg = LogisticRegression(max_iter=10000)
rf = RandomForestClassifier()
svc = SVC(probability=True)

# Crear el ensemble
ensemble = VotingClassifier(estimators=[('lr', logreg), ('rf', rf), ('svc', svc)], voting='soft')

# Entrenar y registrar el ensemble
# train_and_log_ensemble(X_train, y_train, X_test, y_test, ensemble, "Ensemble")


In [41]:
for variant_name, X_train_variant in tqdm(X_train_variants.items()):
    y_train_variant = y_train_variants[variant_name]

    # Aplanar las etiquetas
    y_train_variant = y_train_variant.ravel()
    y_test = y_test.ravel()

    train_and_log_ensemble(
    X_train_variant,
    y_train_variant,
    X_test, y_test,
    ensemble,
    run_name="Ensemble_" + variant_name)

  0%|          | 0/6 [00:00<?, ?it/s]C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\crist\AppData\Roaming\Python\Python39\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [ ]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from lightgbm import LGBMClassifier
import mlflow

# Definir los diferentes parámetros para cada modelo LGBM
params1 = {'learning_rate': 0.05, 'num_leaves': 31, 'max_depth': -1}
params2 = {'learning_rate': 0.01, 'num_leaves': 31, 'max_depth': 5}
params3 = {'learning_rate': 0.1, 'num_leaves': 50, 'max_depth': 5}

# Crear los diferentes modelos LGBM
lgbm1 = LGBMClassifier(**params1)
lgbm2 = LGBMClassifier(**params2)
lgbm3 = LGBMClassifier(**params3)

# Crear el VotingClassifier
model = VotingClassifier(estimators=[('lgbm1', lgbm1), ('lgbm2', lgbm2), ('lgbm3', lgbm3)], voting='soft')

def train_and_log_ensemble(X_train, y_train, X_test, y_val, ensemble, run_name="Ensemble"):
    with mlflow.start_run(run_name=run_name):
        # Entrenar el ensemble
        ensemble.fit(X_train, y_train)

        # Calcular y registrar la métrica de precisión en el conjunto de entrenamiento
        train_preds = np.round(ensemble.predict(X_train))
        train_accuracy = accuracy_score(y_train, train_preds)
        mlflow.log_metric("train_accuracy", train_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de entrenamiento
        train_probs = ensemble.predict_proba(X_train)[:, 1]
        train_roc_auc = roc_auc_score(y_train, train_probs)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Calcular y registrar la métrica de precisión en el conjunto de validación
        val_preds = np.round(ensemble.predict(X_test))
        val_accuracy = accuracy_score(y_val, val_preds)
        mlflow.log_metric("val_accuracy", val_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de validación
        val_probs = ensemble.predict_proba(X_test)[:, 1]
        val_roc_auc = roc_auc_score(y_val, val_probs)
        mlflow.log_metric("val_roc_auc", val_roc_auc)

        # Registrar el modelo
        mlflow.sklearn.log_model(ensemble, run_name)

        print(f"ROC AUC en el conjunto de validación: {val_roc_auc:.4f}")

        return val_roc_auc

# Entrenar y registrar el ensemble
train_and_log_ensemble(X_train, y_train, X_test, y_test, model)


In [ ]:
from mlens.ensemble import SuperLearner
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score
import mlflow

def train_and_log_superlearner(X_train, y_train, X_test, y_val, ensemble, ensemble_name):
    with mlflow.start_run(run_name=ensemble_name):

        # Entrenar el superlearner
        ensemble.fit(X_train, y_train)

        # Calcular y registrar la métrica de precisión en el conjunto de entrenamiento
        train_preds = np.round(ensemble.predict(X_train))
        train_accuracy = accuracy_score(y_train, train_preds)
        mlflow.log_metric("train_accuracy", train_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de entrenamiento
        train_probs = ensemble.predict_proba(X_train)[:, 1]
        train_roc_auc = roc_auc_score(y_train, train_probs)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Calcular y registrar la métrica de precisión en el conjunto de validación
        val_preds = np.round(ensemble.predict(X_test))
        val_accuracy = accuracy_score(y_val, val_preds)
        mlflow.log_metric("val_accuracy", val_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de validación
        val_probs = ensemble.predict_proba(X_test)[:, 1]
        val_roc_auc = roc_auc_score(y_val, val_probs)
        mlflow.log_metric("val_roc_auc", val_roc_auc)

        # Registrar el modelo
        mlflow.sklearn.log_model(ensemble, ensemble_name)

        print(f"ROC AUC en el conjunto de validación: {val_roc_auc:.4f}")

        return val_roc_auc

# Definir los modelos individuales
logreg = LogisticRegression()
rf = RandomForestClassifier()
svc = SVC(probability=True)
mlp = MLPClassifier()
xgb = XGBClassifier()
lgbm = lgb.LGBMClassifier()

# Crear el superlearner
ensemble = SuperLearner(scorer=accuracy_score, random_state=42, verbose=2)
ensemble.add([logreg, rf, svc, mlp, xgb])
ensemble.add_meta(lgbm)

# Entrenar y registrar el superlearner
train_and_log_superlearner(X_train, y_train, X_test, y_test, ensemble, "SuperLearning")
